Certificate      101
Meta_score       157
Gross            169

# Enriquecendo o dataframe 
 - Puxando informações da API TMDB: Popularidade, detalhde de data, produtoras, palavras chaves e diversas outras variaveis que irão auxiliar na análise exploratória 
 - Criação de features financeiras robustas para AED 
 - Adição de certificados e datas de lançamentos manuais por meio de remap

In [2]:
import pandas as pd
import numpy as np

In [7]:
path = '../data/processed/pre_feature_eng.csv'
df = pd.read_csv(path)

In [10]:
df = df[['Series_Title', 'Released_Year']]
df.head()

,Series_Title,Released_Year
0,The Godfather,1972
1,The Dark Knight,2008
2,The Godfather: Part II,1974
3,12 Angry Men,1957
4,The Lord of the Rings: The Return of the King,2003


Adicionando ano de lançamento detalhado, palavras-chave e diminuindo o numero de valores nulos na coluna de certificados

In [11]:
import os
import requests
from dotenv import load_dotenv
import time
import numpy as np
load_dotenv()

API_KEY_TMDB = os.getenv("API_KEY_TMDB")

countrys = ["US", "GB", "CA", "FR", "JP", "DE", "ES", "IT", "BR"]

BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {
    "accept": "application/json",
    "Authorization": f"Bearer {API_KEY_TMDB}"
}

In [12]:
def find_id(title: str, year: int, imdb_id=False):

    url = f"{BASE_URL}/search/movie"
    params = {
        "query": title,
        "year": year,
        "language": "en-US",
        "page": 1
    }
    
    try:
        response = requests.get(url, params=params, headers=HEADERS) 
        response.raise_for_status()
        
        data = response.json()
        results = data.get("results", [])
        
        if results:
            tmdb_id = results[0].get("id")
            
            if imdb_id:
                url = f"{BASE_URL}/movie/{tmdb_id}/external_ids"
                response = requests.get(url, headers=HEADERS)
                response.raise_for_status()
                return response.json().get("imdb_id")
            
            return tmdb_id
        else:
            return None
            
    except requests.RequestException as e:
        print(f"  -> Erro ao buscar ID no TMDB para '{title}': {e}")
        return None

Testando funcionamento da função de encontrar id (IMDB ou TMDB)

In [14]:
display(find_id("Fight Club", 1999))
display(find_id("Fight Club", 1999, imdb_id=True))

550

'tt0137523'

In [16]:
from dateutil import parser


def get_release_certificate(movie_id: int):
    url = f"{BASE_URL}/movie/{movie_id}/release_dates"
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Erro na requisição para o filme ID {movie_id}: {e}")
        return None, None

    results = response.json().get("results", [])
    if not results:
        return None, None

    certification = None
    releases_by_country = {r["iso_3166_1"]: r["release_dates"] for r in results}

    for country in countrys:
        if country in releases_by_country:
            for release in releases_by_country[country]:
                cert = release.get("certification")
                if cert:
                    certification = cert
                    break
        if certification:
            break

    all_dates = [
        release["release_date"]
        for country_releases in releases_by_country.values()
        for release in country_releases
        if release.get("release_date")
    ]

    if all_dates:
        parsed_dates = [parser.parse(d) for d in all_dates]
        release_date = min(parsed_dates).strftime("%Y-%m-%d")
        return certification, release_date
    
    return certification, None


    

In [19]:
def get_keywords(id_movie):
    url = f"{BASE_URL}/movie/{id_movie}/keywords"

    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        data = response.json()
    
        list_keywords = [i['name'] for i in data.get('keywords', [])]

    except requests.RequestException as e:
        print(f"Erro ao buscar keywords para o filme ID {id_movie}: {e}")
        return []

    return list_keywords

In [21]:
print(get_release_certificate(550))
print(get_keywords(550))

('R', '1999-09-10')
['dual identity', 'rage and hate', 'based on novel or book', 'nihilism', 'fight', 'support group', 'dystopia', 'insomnia', 'alter ego', 'breaking the fourth wall', 'split personality', 'quitting a job', 'dissociative identity disorder', 'self destructiveness']


In [27]:
def fill_cer_date_keyw(df: pd.DataFrame, inicio=0) -> pd.DataFrame:
    df_copy = df.copy()
    df_copy['Keywords'] = None
    df_copy['date_details'] = None
    df_copy['Certificate'] = None
    for index in range(inicio, len(df_copy)):
        title = df_copy.loc[index, "Series_Title"]
        year = df_copy.loc[index, "Released_Year"]

        print(f"\nProcessando: '{title}' ({year})")
        
        
        try:
            movie_id = find_id(title, year)
        except Exception as e:
            print(f"Erro ao buscar ID para '{title}': {e}")
            continue

        if not movie_id:
            print(f"não encontrou id {title}")
            continue
        

        keywords = get_keywords(movie_id)
        df_copy.at[index, 'Keywords'] = keywords

        try:
            cert, date = get_release_certificate(movie_id)
        except Exception as e:
            print(f"Erro ao buscar informações de lançamento para '{title}': {e}")
            return df_copy, index

        if cert:
            df_copy.at[index, "Certificate"] = cert
            print(f"certificação atualizada: {cert}")

        if date:
            df_copy.at[index, "date_details"] = date
            print(f"data de lançamento atualizada: {date}")

    return df_copy

df = fill_cer_date_keyw(df, inicio=0)

In [36]:
missing_data_fill_dict = {
    "Soorarai Pottru": {
        "Keywords": "['biography', 'based on book', 'entrepreneur', 'low-cost airline']",
        "date_details": "2020-11-12",
        "Certificate": "U"
    },
    "Vikram Vedha": {
        "Keywords": "['neo-noir', 'gangster', 'police', 'vikramaditya', 'vedhalam']",
        "date_details": "2017-07-21",
        "Certificate": "U/A"
    },
    "Anand": {
        "Keywords": "['terminal illness', 'cancer', 'friendship', 'doctor', 'positive thinking']",
        "date_details": "1971-03-12",
        "Certificate": "U"
    },
    "Tumbbad": {
        "Keywords": "['horror', 'mythology', 'greed', 'treasure', 'monster']",
        "date_details": "2018-10-12",
        "Certificate": "A"
    },
    "Drishyam": {
        "Keywords": "['remake', 'murder', 'cover-up', 'family', 'police investigation']",
        "date_details": "2013-12-19",
        "Certificate": "U/A"
    },
    "Babam ve Oglum": {
        "Keywords": "['family drama', 'father son relationship', 'turkey', 'coup d\\'état']",
        "date_details": "2005-11-18",
        "Certificate": "NR"
    },
    "Uri: The Surgical Strike": {
        "Keywords": "['surgical strike', 'indian army', 'terrorism', 'based on true story']",
        "date_details": "2019-01-11",
        "Certificate": "U/A"
    },
    "Talvar": {
        "Keywords": "['murder investigation', 'based on true story', 'noida double murder case']",
        "date_details": "2015-10-02",
        "Certificate": "U/A"
    },
    "Queen": {
        "Keywords": "['self-discovery', 'honeymoon', 'travel', 'female protagonist', 'coming of age']",
        "date_details": "2014-03-07",
        "Certificate": "U/A"
    },
    "Paan Singh Tomar": {
        "Keywords": "['biography', 'athlete', 'dacoit', 'indian army', 'steeplechase']",
        "date_details": "2012-03-02",
        "Certificate": "U/A"
    },
    "Hera Pheri": {
        "Keywords": "['comedy', 'kidnapping', 'wrong number', 'mistaken identity']",
        "date_details": "2000-03-31",
        "Certificate": "U"
    },
    "Andaz Apna Apna": {
        "Keywords": "['comedy', 'mistaken identity', 'love triangle', 'kidnapping']",
        "date_details": "1994-04-11",
        "Certificate": "U"
    },
    "Yôjinbô": {
        "Keywords": "['samurai', 'ronin', 'feudal japan', 'jidaigeki', 'masterpiece']",
        "date_details": "1961-04-25",
        "Certificate": "NR"
    },
    "Pink": {
        "Keywords": "['courtroom drama', 'consent', 'women\\'s rights', 'false accusation']",
        "date_details": "2016-09-16",
        "Certificate": "U/A"
    },
    "OMG: Oh My God!": {
        "Keywords": "['satire', 'religion', 'god', 'lawsuit', 'earthquake', 'based on play']",
        "date_details": "2012-09-28",
        "Certificate": "U"
    },
    "Barfi!": {
        "Keywords": "['deaf-mute', 'autism', 'love triangle', 'comedy', 'drama']",
        "date_details": "2012-09-14",
        "Certificate": "U"
    },
    "Kahaani": {
        "Keywords": "['mystery', 'thriller', 'pregnant woman', 'kolkata', 'missing husband']",
        "date_details": "2012-03-09",
        "Certificate": "U/A"
    },
    "Zindagi Na Milegi Dobara": {
        "Keywords": "['road trip', 'friendship', 'spain', 'bachelor party', 'coming of age']",
        "date_details": "2011-07-15",
        "Certificate": "U/A"
    },
    "A Wednesday": {
        "Keywords": "['thriller', 'common man', 'terrorism', 'bomb threat', 'police']",
        "date_details": "2008-09-05",
        "Certificate": "U/A"
    },
    "Munna Bhai M.B.B.S.": {
        "Keywords": "['comedy', 'gangster', 'doctor', 'medical school', 'hospital']",
        "date_details": "2003-12-19",
        "Certificate": "U"
    },
    "Fa yeung nin wah": {
        "Keywords": "['romance', 'unconsummated love', 'hong kong', '1960s', 'neighbor']",
        "date_details": "2000-09-29",
        "Certificate": "PG"
    },
    "Badhaai ho": {
        "Keywords": "['social comedy', 'pregnancy', 'middle-aged parents', 'family']",
        "date_details": "2018-10-18",
        "Certificate": "U/A"
    },
    "Baby": {
        "Keywords": "['espionage', 'counter-terrorism', 'secret mission', 'indian intelligence']",
        "date_details": "2015-01-23",
        "Certificate": "U/A"
    },
    "Yip Man": {
        "Keywords": "['martial arts', 'biography', 'wing chun', 'sino-japanese war']",
        "date_details": "2008-12-12",
        "Certificate": "R"
    },
    "Nefes: Vatan Sagolsun": {
        "Keywords": "['war', 'turkish army', 'pkk', 'military outpost', 'turkey']",
        "date_details": "2009-10-16",
        "Certificate": "NR"
    },
    "Mar adentro": {
        "Keywords": "['euthanasia', 'quadriplegic', 'based on true story', 'spain']",
        "date_details": "2004-09-03",
        "Certificate": "PG-13"
    },
    "Kôkaku Kidôtai": {
        "Keywords": "['cyberpunk', 'anime', 'cyborg', 'artificial intelligence', 'philosophy']",
        "date_details": "1995-11-18",
        "Certificate": "NR"
    },
    "Out of the Past": {
        "Keywords": "['film noir', 'femme fatale', 'private eye', 'flashback', 'betrayal']",
        "date_details": "1947-11-25",
        "Certificate": "Approved"
    },
    "Arsenic and Old Lace": {
        "Keywords": "['black comedy', 'screwball comedy', 'murder', 'family', 'poison']",
        "date_details": "1944-09-23",
        "Certificate": "Approved"
    },
    "Raazi": {
        "Keywords": "['spy', 'thriller', 'based on true story', 'indo-pakistani war', 'patriotism']",
        "date_details": "2018-05-11",
        "Certificate": "U/A"
    },
    "M.S. Dhoni: The Untold Story": {
        "Keywords": "['biography', 'cricket', 'sports', 'indian cricket team']",
        "date_details": "2016-09-30",
        "Certificate": "U"
    },
    "Vicky Donor": {
        "Keywords": "['social comedy', 'sperm donor', 'infertility', 'delhi']",
        "date_details": "2012-04-20",
        "Certificate": "U/A"
    },
    "Once": {
        "Keywords": "['musical', 'romance', 'dublin', 'street musician', 'singer-songwriter']",
        "date_details": "2007-03-23",
        "Certificate": "R"
    },
    "Der Name der Rose": {
        "Keywords": "['mystery', 'middle ages', 'monastery', 'monk', 'based on novel']",
        "date_details": "1986-09-24",
        "Certificate": "R"
    },
    "Batoru rowaiaru": {
        "Keywords": "['dystopian', 'survival', 'high school students', 'island', 'cult film']",
        "date_details": "2000-12-16",
        "Certificate": "R15+"
    }
}

for i in df[df['Certificate'].isna() | df['date_details'].isna() | df['Keywords'].isna()].index:
    df.loc[i, 'Keywords'] = missing_data_fill_dict.get(df.loc[i, 'Series_Title'], {}).get('Keywords', np.nan)
    df.loc[i, 'date_details'] = missing_data_fill_dict.get(df.loc[i, 'Series_Title'], {}).get('date_details', np.nan)
    df.loc[i, 'Certificate'] = missing_data_fill_dict.get(df.loc[i, 'Series_Title'], {}).get('Certificate', np.nan)

In [37]:
df.isna().sum()

Series_Title     0
Released_Year    0
Keywords         0
date_details     0
Certificate      0
dtype: int64

In [38]:

df.to_csv('../data/processed/check_2_processing.csv', index=False)

In [42]:
from dateutil import parser

def get_details_movie(title: str, year: int):
    search_url = f"{BASE_URL}/search/movie"
    params = {"query": title, "year": year, "language": "en-US", "page": 1}
    
    try:
        response = requests.get(search_url, params=params, headers=HEADERS)
        response.raise_for_status()
        data = response.json()
        results = data.get("results", [])

        if not results:
            return None

        tmdb_id = None
        for movie in results:
            if movie.get("title", "").lower() == title.lower():
                tmdb_id = movie.get("id")
                break
        
        if not tmdb_id:
            tmdb_id = results[0].get("id")

        if not tmdb_id:
            return None

        details_url = f"{BASE_URL}/movie/{tmdb_id}"
        details_params = {"append_to_response": "release_dates,keywords"}
        
        details_response = requests.get(details_url, params=details_params, headers=HEADERS)
        details_response.raise_for_status()
        details = details_response.json()

        certification = None
        release_date = None
        release_info = details.get("release_dates", {}).get("results", [])
        
        if release_info:
            releases_by_country = {r["iso_3166_1"]: r["release_dates"] for r in release_info}
            for country in countrys:
                if country in releases_by_country:
                    for release in releases_by_country[country]:
                        cert = release.get("certification")
                        if cert and cert.strip():
                            certification = cert
                            break
                if certification:
                    break
            
            all_dates = [
                release["release_date"]
                for country_releases in releases_by_country.values()
                for release in country_releases
                if release.get("release_date")
            ]
            if all_dates:
                parsed_dates = [parser.parse(d) for d in all_dates]
                if parsed_dates:
                    release_date = min(parsed_dates).strftime("%Y-%m-%d")

        return {
            "runtime": details.get("runtime"),
            "original_language": details.get("original_language"),
            "production_companies": [c["name"] for c in details.get("production_companies", [])],
            "production_countries": [c["name"] for c in details.get("production_countries", [])],
            "budget": details.get("budget"),
            "revenue": details.get("revenue"),
            "popularity": details.get("popularity"),
            "keywords": [k["name"] for k in details.get("keywords", {}).get("keywords", [])],
            "certificate": certification,
            "release_date": release_date
        }

    except requests.RequestException:
        return None

In [43]:
display(get_details_movie("Fight Club", 1999))

{'runtime': 139,
 'original_language': 'en',
 'production_companies': ['Fox 2000 Pictures',
  'Regency Enterprises',
  'Linson Entertainment',
  '20th Century Fox',
  'Taurus Film'],
 'production_countries': ['Germany', 'United States of America'],
 'budget': 63000000,
 'revenue': 100853753,
 'popularity': 25.5065,
 'keywords': ['dual identity',
  'rage and hate',
  'based on novel or book',
  'nihilism',
  'fight',
  'support group',
  'dystopia',
  'insomnia',
  'alter ego',
  'breaking the fourth wall',
  'split personality',
  'quitting a job',
  'dissociative identity disorder',
  'self destructiveness'],
 'certificate': 'R',
 'release_date': '1999-09-10'}

In [46]:
def enrich_row(row: pd.Series) -> pd.Series:
    title = row.get("Series_Title")
    year = row.get("Released_Year")

    print(f"\nProcessando: '{title}' ({year})")

    movie_data = get_details_movie(title, year)

    if movie_data:
        row['Budget'] = movie_data.get('budget')
        row['Revenue'] = movie_data.get('revenue')
        row['Popularity'] = movie_data.get('popularity')
        row['Original_Language'] = movie_data.get('original_language')
        row['Production_Companies'] = movie_data.get('production_companies')
        row['Production_Countries'] = movie_data.get('production_countries')

        if pd.isna(row.get('Gross')) and movie_data.get('revenue') is not None:
            row['Gross'] = movie_data['revenue']
            print(f"  -> Coluna 'Gross' preenchida com revenue: {movie_data['revenue']}")

    return row

def enrich_df(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy()

    new_columns = [
        'Budget', 'Revenue', 'Popularity',
        'Original_Language', 'Production_Companies', 'Production_Countries'
    ]
    for col in new_columns:
        if col not in df_copy.columns:
            df_copy[col] = pd.NA

    df_copy = df_copy.apply(enrich_row, axis=1)

    print("\nProcesso de enriquecimento de dados concluído.")
    return df_copy


In [48]:
df = enrich_df(df)
df.to_csv('../data/processed/check_3_processing.csv', index=False)


Processando: 'The Godfather' (1972)
  -> Coluna 'Gross' preenchida com revenue: 245066411

Processando: 'The Dark Knight' (2008)
  -> Coluna 'Gross' preenchida com revenue: 1004558444

Processando: 'The Godfather: Part II' (1974)
  -> Coluna 'Gross' preenchida com revenue: 102600000

Processando: '12 Angry Men' (1957)
  -> Coluna 'Gross' preenchida com revenue: 4360000

Processando: 'The Lord of the Rings: The Return of the King' (2003)
  -> Coluna 'Gross' preenchida com revenue: 1118888979

Processando: 'Pulp Fiction' (1994)
  -> Coluna 'Gross' preenchida com revenue: 213928762

Processando: 'Schindler's List' (1993)
  -> Coluna 'Gross' preenchida com revenue: 321365567

Processando: 'Inception' (2010)
  -> Coluna 'Gross' preenchida com revenue: 839030630

Processando: 'Fight Club' (1999)
  -> Coluna 'Gross' preenchida com revenue: 100853753

Processando: 'The Lord of the Rings: The Fellowship of the Ring' (2001)
  -> Coluna 'Gross' preenchida com revenue: 871368364

Processando: 'Fo

In [51]:
if 'Gross' in df.columns:
    df.drop(columns=['Gross'], inplace=True)

In [56]:

financial_cols = ['Budget', 'Revenue']
for col in financial_cols:
    df[col] = df[col].replace(0, np.nan)

In [57]:
df.isnull().sum()

Budget                  155
Certificate               0
Keywords                  0
Original_Language         6
Popularity                6
Production_Companies      6
Production_Countries      6
Released_Year             0
Revenue                 117
Series_Title              0
date_details              0
dtype: int64

Arrumando detalhes de formatação para df final

In [53]:
import ast

def safe_literal_eval(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except (ValueError, SyntaxError):

            return val

    return val

cols_to_convert = ['Keywords', 'Production_Companies', 'Production_Countries']
for col in cols_to_convert:
    df[col] = df[col].apply(safe_literal_eval)


Columns successfully converted.


In [68]:
df.to_csv('../data/processed/check_4_processing.csv', index=False)

In [132]:
df = pd.read_csv('../data/processed/check_4_processing.csv')
pre_processed_df = pd.read_csv('../data/processed/pre_feature_eng.csv')
df.head()

,Budget,Certificate,Keywords,Original_Language,Popularity,Production_Companies,Production_Countries,Released_Year,Revenue,Series_Title,date_details
0,6000000.0,R,"['based on novel or book', 'loss of loved one'...",en,26.3622,"['Paramount Pictures', 'Alfran Productions']",['United States of America'],1972,2.450664e+08,The Godfather,1972-03-14
1,185000000.0,PG-13,"['sadism', 'chaos', 'secret identity', 'crime ...",en,29.9949,"['Warner Bros. Pictures', 'Legendary Pictures'...","['United Kingdom', 'United States of America']",2008,1.004558e+09,The Dark Knight,2008-07-14
2,13000000.0,R,"[""new year's eve"", 'new york city', 'based on ...",en,16.6569,"['Paramount Pictures', 'The Coppola Company']",['United States of America'],1974,1.026000e+08,The Godfather: Part II,1974-12-12
3,397751.0,NR,"['death penalty', 'anonymity', 'court case', '...",en,12.3449,"['United Artists', 'Orion-Nova Productions']",['United States of America'],1957,4.360000e+06,12 Angry Men,1957-04-10
4,94000000.0,PG-13,"['army', 'based on novel or book', 'elves', 'd...",en,23.6182,"['New Line Cinema', 'WingNut Films', 'The Saul...","['New Zealand', 'United States of America']",2003,1.118889e+09,The Lord of the Rings: The Return of the King,2003-12-01


In [134]:
display(pre_processed_df['Certificate'].value_counts())
display(df['Certificate'].value_counts())

mask = (
    (pre_processed_df['Certificate'] == 'Unrated') |
    (pre_processed_df['Certificate'].isna()) |
    (pre_processed_df['Certificate'] == 'NR')
)

pre_processed_df.loc[mask, 'Certificate'] = df.loc[mask, 'Certificate'].values



Certificate
U           234
A           196
UA          175
R           146
PG-13        43
PG           37
Passed       34
G            12
Approved     11
TV-PG         3
GP            2
TV-14         1
Unrated       1
TV-MA         1
16            1
U/A           1
Name: count, dtype: int64

Certificate
R           379
PG-13       178
PG          165
NR          144
G            51
15           19
U            15
U/A          14
12            8
NC-17         4
TP            4
16            3
R15+          3
18            3
6             2
R18+          2
Approved      2
A             1
12A           1
14            1
Name: count, dtype: int64

In [135]:
certificate_full_rank_map = {
    'Unrated': 0,
    'NR': 0,

    'G': 1,
    'U': 1,
    'Passed': 1,
    'Approved': 1,
    'TP': 1,

    'PG': 2,
    'GP': 2,       
    'TV-PG': 2,
    '6': 2,


    'PG-13': 3,
    'UA': 3,       
    'U/A': 3,
    'TV-14': 3,
    '12': 3,
    '12A': 3,
    '14': 3,

    'R': 4,
    '15': 4,
    '16': 4,
    'R15+': 4,


    'NC-17': 5,
    'A': 5,
    'TV-MA': 5,
    '18': 5,
    'R18+': 5
}

df['Certificate_ord'] = df['Certificate'].map(certificate_full_rank_map)
pre_processed_df['Certificate_ord'] = pre_processed_df['Certificate'].map(certificate_full_rank_map)


In [136]:
pre_processed_df['Certificate_ord'] = pre_processed_df['Certificate_ord'].fillna(df['Certificate_ord'])
pre_processed_df['Certificate'] = pre_processed_df['Certificate'].fillna(df['Certificate'])

In [137]:
display(df.columns)

display(pre_processed_df.columns)

Index(['Budget', 'Certificate', 'Keywords', 'Original_Language', 'Popularity',
       'Production_Companies', 'Production_Countries', 'Released_Year',
       'Revenue', 'Series_Title', 'date_details', 'Certificate_ord'],
      dtype='object')

Index(['Series_Title', 'Released_Year', 'Certificate', 'Runtime', 'Genre',
       'IMDB_Rating', 'Overview', 'Meta_score', 'Director', 'Star1', 'Star2',
       'Star3', 'Star4', 'No_of_Votes', 'Gross', 'Certificate_ord'],
      dtype='object')

In [138]:
pre_processed_df['Budget'] = df['Budget']
pre_processed_df['Revenue'] = df['Revenue']
pre_processed_df['Popularity'] = df['Popularity']
pre_processed_df['Original_Language'] = df['Original_Language']
pre_processed_df['Production_Companies'] = df['Production_Companies']
pre_processed_df['Production_Countries'] = df['Production_Countries']
pre_processed_df['Keywords'] = df['Keywords']
pre_processed_df['date_details'] = df['date_details']


In [139]:
fator = pre_processed_df['Gross'].mean() / pre_processed_df['Revenue'].mean()
pre_processed_df['Gross_fill'] = pre_processed_df['Gross'].fillna(pre_processed_df['Revenue'] * fator)
pre_processed_df['Gross_fill'] = pre_processed_df['Gross_fill'].fillna(pre_processed_df['Gross'].median())

print(fator)

0.4423645977977842


In [140]:
factor_med = (pre_processed_df['Revenue'] / pre_processed_df['Budget']).median()
pre_processed_df['Budget'] = pre_processed_df['Budget'].fillna(pre_processed_df['Revenue'] / factor_med)
pre_processed_df['Revenue'] = pre_processed_df['Revenue'].fillna(pre_processed_df['Budget'] * factor_med)


In [141]:
pre_processed_df.isna().sum().sort_values(ascending=False)

Gross                   169
Meta_score              157
Budget                   85
Revenue                  85
Popularity                6
Original_Language         6
Production_Countries      6
Production_Companies      6
Series_Title              0
Director                  0
Overview                  0
IMDB_Rating               0
Genre                     0
Certificate               0
Runtime                   0
Released_Year             0
Star1                     0
Certificate_ord           0
No_of_Votes               0
Star3                     0
Star4                     0
Star2                     0
Keywords                  0
date_details              0
Gross_fill                0
dtype: int64

In [142]:
pre_processed_df

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,...,Certificate_ord,Budget,Revenue,Popularity,Original_Language,Production_Companies,Production_Countries,Keywords,date_details,Gross_fill
0,The Godfather,1972,A,175,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,...,5,6000000.0,2.450664e+08,26.3622,en,"['Paramount Pictures', 'Alfran Productions']",['United States of America'],"['based on novel or book', 'loss of loved one'...",1972-03-14,1.349664e+08
1,The Dark Knight,2008,UA,152,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,...,3,185000000.0,1.004558e+09,29.9949,en,"['Warner Bros. Pictures', 'Legendary Pictures'...","['United Kingdom', 'United States of America']","['sadism', 'chaos', 'secret identity', 'crime ...",2008-07-14,5.348584e+08
2,The Godfather: Part II,1974,A,202,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,...,5,13000000.0,1.026000e+08,16.6569,en,"['Paramount Pictures', 'The Coppola Company']",['United States of America'],"[""new year's eve"", 'new york city', 'based on ...",1974-12-12,5.730000e+07
3,12 Angry Men,1957,U,96,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,...,1,397751.0,4.360000e+06,12.3449,en,"['United Artists', 'Orion-Nova Productions']",['United States of America'],"['death penalty', 'anonymity', 'court case', '...",1957-04-10,4.360000e+06
4,The Lord of the Rings: The Return of the King,2003,U,201,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,...,1,94000000.0,1.118889e+09,23.6182,en,"['New Line Cinema', 'WingNut Films', 'The Saul...","['New Zealand', 'United States of America']","['army', 'based on novel or book', 'elves', 'd...",2003-12-01,3.778459e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,Breakfast at Tiffany's,1961,A,115,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,...,5,2500000.0,9.500000e+06,4.8948,en,"['Paramount Pictures', 'Jurow-Shepherd']",['United States of America'],"['new york city', ""love of one's life"", 'broke...",1961-10-05,4.202464e+06
995,Giant,1956,G,201,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,...,1,5400000.0,3.285582e+07,3.1848,en,"['Warner Bros. Pictures', 'George Stevens Jr. ...",['United States of America'],"['hotel', 'petrol', 'jealousy', 'judge', 'fune...",1956-10-10,1.453425e+07
996,From Here to Eternity,1953,Passed,118,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,...,1,1650000.0,3.050000e+07,4.0353,en,['Columbia Pictures'],['United States of America'],"['beach', 'based on novel or book', 'hawaii', ...",1953-08-05,3.050000e+07
997,Lifeboat,1944,NR,97,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,...,0,1590000.0,1.000000e+06,1.9989,en,['20th Century Fox'],['United States of America'],"['sea', 'journalist', 'submarine', 'boat', 'wo...",1944-01-11,4.423646e+05


In [143]:
certificate_mapping = {
    'U': 'G', 'Approved': 'G', 'Passed': 'G', 'TP': 'G',
    '6': 'PG',
    'UA': 'PG-13', 'U/A': 'PG-13', '12': 'PG-13', '12A': 'PG-13', '14': 'PG-13',
    '15': 'R', '16': 'R', 'R15+': 'R',
    'A': 'NC-17', '18': 'NC-17', 'R18+': 'NC-17',
    'NR': 'Unrated'
}
pre_processed_df['Certificate'] = pre_processed_df['Certificate'].map(certificate_mapping).fillna('Unrated')

In [144]:
df['Certificate'].value_counts()

Certificate
R           379
PG-13       178
PG          165
NR          144
G            51
15           19
U            15
U/A          14
12            8
NC-17         4
TP            4
16            3
R15+          3
18            3
6             2
R18+          2
Approved      2
A             1
12A           1
14            1
Name: count, dtype: int64

In [149]:
pre_processed_df['Certificate'].value_counts()

Certificate
Unrated    322
G          286
NC-17      198
PG-13      181
R           12
Name: count, dtype: int64

In [146]:
pre_processed_df.describe()

,Released_Year,Runtime,IMDB_Rating,Meta_score,No_of_Votes,Gross,Certificate_ord,Budget,Revenue,Popularity,Gross_fill
count,999.000000,999.000000,999.000000,842.000000,9.990000e+02,8.300000e+02,999.000000,9.140000e+02,9.140000e+02,993.000000,9.990000e+02
mean,1991.218218,122.871872,7.947948,77.969121,2.716214e+05,6.808257e+07,2.740741,2.837943e+07,1.492503e+08,6.658564,5.897940e+07
std,23.297166,28.101227,0.272290,12.383257,3.209126e+05,1.098076e+08,1.636217,4.838764e+07,2.838934e+08,5.725451,1.021981e+08
min,1920.000000,45.000000,7.600000,28.000000,2.508800e+04,1.305000e+03,0.000000,1.050000e+02,3.193000e+03,0.007900,1.305000e+03
25%,1976.000000,103.000000,7.700000,70.000000,5.547150e+04,3.245338e+06,1.000000,2.555600e+06,9.031733e+06,2.779600,3.129808e+06
50%,1999.000000,119.000000,7.900000,79.000000,1.383560e+05,2.345744e+07,3.000000,9.000000e+06,4.002362e+07,4.654800,2.245598e+07
75%,2009.000000,137.000000,8.100000,87.000000,3.731675e+05,8.087634e+07,4.000000,2.875000e+07,1.500000e+08,8.925500,6.157656e+07
max,2020.000000,321.000000,9.200000,100.000000,2.303232e+06,9.366622e+08,5.000000,3.560000e+08,2.923706e+09,47.132600,9.366622e+08


In [147]:
pre_processed_df['Brute_Profit'] = pre_processed_df['Revenue'] - pre_processed_df['Budget']
pre_processed_df['ROI'] = pre_processed_df['Brute_Profit'] / pre_processed_df['Budget']

In [153]:
#timestamp 1970

pre_processed_df['Timestamp'] = pd.to_datetime(pre_processed_df['date_details'], errors='coerce')
pre_processed_df['Timestamp'] = pre_processed_df['Timestamp'].astype('int64') // 10**9
pre_processed_df['Timestamp'].value_counts()

Timestamp
 1347062400    3
 1157068800    3
 1400198400    3
 863568000     2
 1453507200    2
              ..
-260064000     1
-417398400     1
-517795200     1
-819676800     1
 1215993600    1
Name: count, Length: 948, dtype: int64

In [ ]:
df

In [154]:
pre_processed_df.to_csv('../data/processed/final_df.csv', index=False)